### 构造可以生成 x state 的变分量子电路

In [1]:
# 导入必要的库
# from jax import numpy as jnp
# import jax    # 机器学习库
# import optax  # 优化器
import matplotlib.pyplot as plt

import pennylane as qml

from pennylane import numpy as np
# from qasm_until import export_to_openqasm

In [2]:
A_matrix = np.array([[2, 5, -13],
              [1, -3, 1],
              [-5, 6, 8]]).astype(float)

# 将矩阵 A 变成一个厄尔米特矩阵
H_matrix = np.block([[np.zeros((3, 3)), A_matrix], [A_matrix.T, np.zeros((3, 3))]])

# 将 H_matrix 变为 8×8，方便用 pauli 算子的线性组合来表示
H_matrix_2 = np.zeros((8, 8))

for i in range(H_matrix.shape[0]):
    for j in range(H_matrix.shape[0]):
        H_matrix_2[i,j] = H_matrix[i,j]

b_vec = np.array([1000, 0, -600]).astype(float)

# 相对应的 b 和 x  也要进行维度拓展，以适应 H_matrixs
b_vec_2 = np.zeros((8))
b_vec_2[:3] = b_vec
b_norm = np.linalg.norm(b_vec_2)


x_vec = np.array([1200, 500, 300]).astype(float)
# 归一化
x_vec_2 = np.zeros((8))
x_vec_2[3:6] = x_vec

# 计算范数
x_norm = np.linalg.norm(x_vec_2)

# 归一化向量，转回行向量，方便后续计算
x_normalized = (x_vec_2 / x_norm)

print(x_normalized)

rho = x_normalized.reshape(-1,1) @ x_normalized.reshape(1,-1)

print(np.trace(rho))
print(np.trace(rho @ rho))

[0.         0.         0.         0.89943803 0.37476584 0.22485951
 0.         0.        ]
1.0
1.0


In [3]:
# 构造设备
# H_matrix_2 只需要 3 qubits，但是后面要变成受控的，所以需要 4 qubits
qubits = 3 
dev = qml.device("default.qubit", wires=qubits)


# 定义一个变分电路
@qml.qnode(dev)
def variational_circuit(params):
    # for i in range(3):
    #     qml.Hadamard(wires=i)
    qml.BasicEntanglerLayers(params, wires=[0, 1, 2], rotation=qml.RY)
    
    return qml.state()

# 定义损失函数
def cost(params):
    prepared_state = variational_circuit(params)
    return 1 - np.abs(np.dot(np.conj(x_normalized), prepared_state))**2


# 初始化参数
np.random.seed(1024)
shape = qml.BasicEntanglerLayers.shape(n_layers=3, n_wires=3)

params = np.random.uniform(low=0, high=2*np.pi, size=shape)
# params = qml_np.array(params, requires_grad=True)

In [4]:
params

tensor([[4.06956402, 6.26379276, 3.25973705],
        [4.13504424, 3.7640268 , 4.73166161],
        [0.85606595, 0.02586861, 0.939392  ]], requires_grad=True)

In [5]:
# 使用优化器
optimizer = qml.GradientDescentOptimizer(stepsize=0.8)
max_iterations = 200
tol = 1e-10

# 优化参数
for i in range(max_iterations):
    params, cost_val = optimizer.step_and_cost(cost, params)
    if cost_val < tol:
        break
    if i % 5 == 0:
        print(f"Iteration {i}: Cost = {cost_val}")

print(f"Final cost: {cost_val}")
print(f"Optimized parameters: {params}")

# 打印最终准备的量子态
final_state = variational_circuit(params)
print("Prepared Quantum State:")
print(final_state)

Iteration 0: Cost = 0.7529527161741065
Iteration 5: Cost = 0.30940376286268645
Iteration 10: Cost = 0.09863564969564032
Iteration 15: Cost = 0.024002527177112154
Iteration 20: Cost = 0.007423023416895158
Iteration 25: Cost = 0.0038722045950307926
Iteration 30: Cost = 0.002599025760140372
Iteration 35: Cost = 0.0018946311699533647
Iteration 40: Cost = 0.0014436455447766061
Iteration 45: Cost = 0.0011345456290454692
Iteration 50: Cost = 0.0009125567824491698
Iteration 55: Cost = 0.0007473140659620681
Iteration 60: Cost = 0.0006207683693628674
Iteration 65: Cost = 0.0005216072283009687
Iteration 70: Cost = 0.00044242679237471716
Iteration 75: Cost = 0.00037820295508306234
Iteration 80: Cost = 0.000325420451822378
Iteration 85: Cost = 0.00028155431447540735
Iteration 90: Cost = 0.00024474918012196145
Iteration 95: Cost = 0.0002136143372882815
Iteration 100: Cost = 0.0001870889279915744
Iteration 105: Cost = 0.00016435102370671029
Iteration 110: Cost = 0.0001447549056872166
Iteration 115: C

In [6]:
# import copy 
# final_state_2 = copy.deepcopy(final_state)
# # print(final_state_2)
# for i in range(len(final_state)):
#     if final_state[i] <= 1e-3:
#         final_state_2[i] = 0
# print(final_state_2)
    

In [7]:
params

tensor([[4.13345498, 6.96129538, 3.15416639],
        [3.8794304 , 4.86300243, 4.68250843],
        [0.41432465, 2.37792433, 1.63689462]], requires_grad=True)

In [8]:
value = np.sqrt((H_matrix_2 @ final_state.real.reshape(-1,1)).conj().T @ (H_matrix_2 @ final_state.real.reshape(-1,1)))

xxx = b_norm * final_state/value

print(xxx.real)

[[ 2.18896962e+00 -1.86724246e+00  6.41774514e-01  1.19293506e+03
   4.96464016e+02  2.97767540e+02  1.29482982e+00 -4.62550634e+00]]


In [9]:
final_state

tensor([ 1.65081049e-03+0.j, -1.40818009e-03+0.j,  4.83993970e-04+0.j,
         8.99651452e-01+0.j,  3.74408121e-01+0.j,  2.24561260e-01+0.j,
         9.76495347e-04+0.j, -3.48832359e-03+0.j], requires_grad=True)

In [10]:
x_normalized

tensor([0.        , 0.        , 0.        , 0.89943803, 0.37476584,
        0.22485951, 0.        , 0.        ], requires_grad=True)

In [11]:
x_vec

tensor([1200.,  500.,  300.], requires_grad=True)